<a href="https://colab.research.google.com/github/xtruffles/Dominion/blob/master/simulations/lemon_rtt_spool_play.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

def randomize(li):
  return np.random.permutation(li).tolist()

def spool_no_discard(hand, deck, discard):
  if 'C' in deck: # draw everything up to the copper
    copper = deck.index('C')
    return hand+deck[0:copper], deck[copper+1:], discard
  else:
    if discard == []:
      return hand+deck, [], []
    # shuffle the discard and draw into it
    discard = randomize(discard)
    # play spool on the remaining discard
    discard_draw, discard_remaining, discard = spool_no_discard([], discard, [])
    return hand+deck+discard_draw, discard_remaining, []

def spool(hand, deck, discard):
  if deck[0] == 'C':
    # add copper to the discard and play pool on the rest
    return spool_no_discard(hand, deck[1:], discard+['C'])
  else:
    return spool_no_discard(hand, deck, discard)

def cantrip(hand, deck, discard, size):
  if len(deck) >= size:
    return hand+deck[0:size], deck[size:], discard 
  # have to draw into discard
  if discard == []:
    return hand+deck, [], [] 
  discard = randomize(discard)
  deck = deck+discard
  return hand + deck[0:size], deck[size:], []

def lab(hand, deck, discard):
  return cantrip(hand, deck, discard, 2)

def village(hand, deck, discard):
  return cantrip(hand, deck, discard, 1)

def play(hand, deck, discard):
  if deck == [] and discard == []:
    return True
  if deck == []:
    discard = randomize(discard)
    return play(hand, discard, [])
  if 'S' in hand:
    hand, deck, discard = spool(hand, deck, discard)
    hand.remove('S')
    return play(hand, deck, discard)
  if 'L' in hand:
    hand, deck, discard = lab(hand, deck, discard)
    hand.remove('L')
    return play(hand, deck, discard)
  if 'V' in hand:
    hand, deck, discard = village(hand, deck, discard)
    hand.remove('V')
    return play(hand, deck, discard)
  return False

def lemonspawn(hand, deck):
  discard = [deck[0]]
  deck = deck[1:]
  for i in range(4):
    if deck[0] in ['S', 'V']:
      discard = discard + [deck[0]]
      deck = deck[1:]
  return play(hand, deck, discard)
    
def rtt(hand, deck):
  discard = []
  for i in range(5):
    if deck[0] in ['S', 'V']:
      discard = discard + [deck[0]]
      deck = deck[1:]
  return play(hand, deck, discard)

# 2 coppers, 2 groves, lab
hand = ['C', 'C', 'G', 'G', 'L']
# 3 coppers, 1 grove, 5 pools, 2 villages ?
deck_copper = ['C', 'C', 'C', 'G', 'S', 'S', 'S', 'S','S','V', 'V']
# start with empty discard
discard = []

trials = 500000
rtt_copper_hit = 0
lemon_copper_hit = 0

for i in range(trials):
  shuffle = ['C'] + randomize(deck_copper)
  if rtt(hand, shuffle):
    rtt_copper_hit += 1
  if lemonspawn(hand, shuffle):
    lemon_copper_hit += 1

rtt_grove_hit = 0
lemon_grove_hit = 0

# 4 coppers, 5 pools, 2 villages ?
deck_grove = ['C', 'C', 'C', 'C', 'S', 'S', 'S', 'S','S','V', 'V']
for i in range(trials):
  shuffle = ['G'] + randomize(deck_grove)
  if rtt(hand, shuffle):
    rtt_grove_hit += 1
  if lemonspawn(hand, shuffle):
    lemon_grove_hit += 1

# 2 coppers, 2 groves, lab
hand = ['C', 'C', 'G', 'G', 'L']
# 3 coppers, 1 grove, 5 pools, 2 villages ?
deck_copper = ['C', 'C', 'C', 'G', 'S', 'S', 'S', 'S','S','V', 'V']
# start with empty discard
discard = []

trials = 500000
rtt_copper_hit = 0
lemon_copper_hit = 0

for i in range(trials):
  shuffle = ['C'] + randomize(deck_copper)
  if rtt(hand, shuffle):
    rtt_copper_hit += 1
  if lemonspawn(hand, shuffle):
    lemon_copper_hit += 1

rtt_normal_hit = 0
lemon_normal_hit = 0

# 4 coppers, 1 grove, 5 pools, 2 villages ?
deck_normal = ['C', 'C', 'C', 'C', 'G', 'S', 'S', 'S', 'S','S','V', 'V']
for i in range(trials):
  shuffle = randomize(deck_normal)
  if rtt(hand, shuffle):
    rtt_normal_hit += 1
  if lemonspawn(hand, shuffle):
    lemon_normal_hit += 1

hit = 0 
deck_normal = ['C', 'C', 'C', 'C', 'G', 'S', 'S', 'S', 'S','S','V', 'V']
for i in range(trials):
  shuffle = randomize(deck_normal)
  if play(hand, shuffle, []):
    hit += 1

print("no attack probability: {}".format(hit/trials))
print("lemon hit with always discard first: {}".format(lemon_normal_hit/trials))
print("rtt hit with discard whenever good: {}".format(rtt_normal_hit/trials))
print("lemon hit with always discard first with grove on top: {}".format(lemon_grove_hit/trials))
print("rtt hit with discard whenever good with grove on top: {}".format(rtt_grove_hit/trials))
print("lemon hit with always discard first with copper on top: {}".format(lemon_copper_hit/trials))
print("rtt hit with discard whenever good with copper on top: {}".format(rtt_copper_hit/trials))


no attack probability: 0.656538
lemon hit with always discard first: 0.365404
rtt hit with discard whenever good: 0.357266
lemon hit with always discard first with grove on top: 0.43548
rtt hit with discard whenever good with grove on top: 0.419774
lemon hit with always discard first with copper on top: 0.4723
rtt hit with discard whenever good with copper on top: 0.453988
